In [ ]:
## Development for this pretty much halted since this version of the project is slow AF. Each step
## of the environment takes at least one minute to perform even on a GPU due to the large enumeration
## space over the cells of the observation. Contiuation of this project is being done on the second version of RIPS.
## Codenamed "STRANGE MATTER", this variant is much faster since convolutional filters are used to decrease the dimensions of the input space.

# Reward Induced Program Synthesis - RIPS 001 "DOPPLER"
!pip install git+https://github.com/ayaz-amin/schema-rl.git

  Cloning https://github.com/ayaz-amin/schema-rl.git to /tmp/pip-req-build-7cum3xdo
  Running command git clone -q https://github.com/ayaz-amin/schema-rl.git /tmp/pip-req-build-7cum3xdo
     |████████████████████████████████| 11.4MB 3.5MB/s 
  Created wheel for schema-games: filename=schema_games-1.0.0-cp36-none-any.whl size=28598 sha256=b78c31991de690cadac6a43a110c2c241ab3bbcd5703415b6d1f25ba66dfd849
  Stored in directory: /tmp/pip-ephem-wheel-cache-d_ujtdql/wheels/4a/fe/a0/6800016926ff46b11b889e96271961e5c5947e7f7e67c99435
Successfully built schema-games


In [ ]:
# Ripped off from https://github.com/cog-isa/schema-rl/

from schema_games.breakout.constants import \
    BRICK_SIZE, ENV_SIZE, DEFAULT_HEIGHT, DEFAULT_WIDTH, DEFAULT_PADDLE_SHAPE


class Constants:
    """
    N: number of entities
    M: number of attributes of each entity
    A: number of available actions
    L: number of schemas
    T: size of look-ahead window
    """

    # --- Agent options ---

    DO_PRELOAD_DUMP_PARAMS = False

    DO_PRELOAD_HANDCRAFTED_ATTRIBUTE_PARAMS = False
    DO_PRELOAD_HANDCRAFTED_REWARD_PARAMS = False

    DO_LEARN_ATTRIBUTE_PARAMS = True
    DO_LEARN_REWARD_PARAMS = True

    # planning options are ('agent', 'hardcoded', 'random')
    PLANNING_TYPE = 'agent'

    LEARNING_PERIOD = 128
    LEARNING_SOLVER = 'cbc'  # 'cbc', 'gurobi'
    USE_EMERGENCY_PLANNING = True

    VISUALIZE_STATE = True
    VISUALIZE_SCHEMAS = False
    VISUALIZE_INNER_STATE = True
    VISUALIZE_BACKTRACKING = True
    LOG_PLANNED_ACTIONS = False

    N_LEARNING_THREADS = 16

    L = 1000
    NEIGHBORHOOD_RADIUS = 2

    if ENV_SIZE == 'SMALL':
        # T = 112 is enough to hit the furthest brick
        # EMERG_PERIOD = 15 is enough to plan with ball being near closest brick
        T = 112  # min 50
        PLANNING_PERIOD = 10  # run planning every *this* steps
        EMERGENCY_PLANNING_PERIOD = 10  # run planning every *this* steps if there are no planned actions
    elif ENV_SIZE == 'DEFAULT':
        T = 130  # min 112
        PLANNING_PERIOD = 10
        EMERGENCY_PLANNING_PERIOD = 30
    else:
        assert False

    # --- Constants ---

    LEARNING_SCHEMA_TOLERANCE = 1e-8
    ADDING_SCHEMA_TOLERANCE = 1e-8

    SCREEN_HEIGHT = DEFAULT_HEIGHT
    SCREEN_WIDTH = DEFAULT_WIDTH
    N = SCREEN_WIDTH * SCREEN_HEIGHT
    M = 5
    N_PREDICTABLE_ATTRIBUTES = M - 1
    ACTION_SPACE_DIM = 3
    REWARD_SPACE_DIM = 2

    FILTER_SIZE = 2 * NEIGHBORHOOD_RADIUS + 1
    NEIGHBORS_NUM = FILTER_SIZE ** 2 - 1
    FRAME_STACK_SIZE = 2
    SCHEMA_VEC_SIZE = FRAME_STACK_SIZE * (M * (NEIGHBORS_NUM + 1)) + ACTION_SPACE_DIM
    TIME_SIZE = FRAME_STACK_SIZE + T
    LEARNING_BATCH_SIZE = FRAME_STACK_SIZE + 1

    # indices of corresponding attributes in entities' vectors
    BALL_IDX = 0
    PADDLE_IDX = 1
    WALL_IDX = 2
    BRICK_IDX = 3
    VOID_IDX = 4
    FAKE_ENTITY_IDX = N

    # action indices
    ACTION_NOP = 0
    ACTION_MOVE_LEFT = 1
    ACTION_MOVE_RIGHT = 2

    ENTITY_NAMES = {
        BALL_IDX: 'BALL',
        PADDLE_IDX: 'PADDLE',
        WALL_IDX: 'WALL',
        BRICK_IDX: 'BRICK',
    }

    REWARD_NAMES = {
        0: 'POSITIVE',
        1: 'NEGATIVE',
    }

    ATTRIBUTE = 'attribute'
    REWARD = 'reward'
    ALLOWED_OBJ_TYPES = {ATTRIBUTE, REWARD}

    DEFAULT_PADDLE_SHAPE = DEFAULT_PADDLE_SHAPE

In [ ]:
# Ripped off from https://github.com/cog-isa/schema-rl/
class EntityExtractor(Constants):
    @classmethod
    def extract(cls, env):
        """
        :param env: object, child of BreakoutEngine
        """
        matrix = np.zeros((cls.N, cls.M), dtype=int)

        for ball in env.balls:
            if ball.is_entity:
                for state, eid in env.parse_object_into_pixels(ball):
                    idx = cls.get_entity_idx(state)
                    matrix[idx][cls.BALL_IDX] = 0

        if env.paddle.is_entity:
            for state, eid in env.parse_object_into_pixels(env.paddle):
                idx = cls.get_entity_idx(state)
                matrix[idx][cls.PADDLE_IDX] = 1

        for wall in env.walls:
            if wall.is_entity:
                for state, eid in env.parse_object_into_pixels(wall):
                    idx = cls.get_entity_idx(state)
                    matrix[idx][cls.WALL_IDX] = 2

        for brick in env.bricks:
            if brick.is_entity:
                for state, eid in env.parse_object_into_pixels(brick):
                    idx = cls.get_entity_idx(state)
                    matrix[idx][cls.BRICK_IDX] = 3

        # raise VOID bit
        void_mask = ~matrix.any(axis=1)
        matrix[void_mask, cls.VOID_IDX] = -1
        matrix = matrix.reshape(cls.M, cls.SCREEN_WIDTH, cls.SCREEN_HEIGHT)
        return matrix.max(0)

    @classmethod
    def transform_pos_to_index(cls, pos):
        return pos[0] * cls.SCREEN_WIDTH + pos[1]

    @classmethod
    def get_entity_pos(cls, state):
        return [*state][0][1]

    @classmethod
    def get_entity_idx(cls, state):
        pos = cls.get_entity_pos(state)
        idx = cls.transform_pos_to_index(pos)
        return idx

    @classmethod
    def get_ball_x(cls, env):
        for ball in env.balls:
            if ball.is_entity:
                for state, eid in env.parse_object_into_pixels(ball):
                    pos = cls.get_entity_pos(state)
                    return pos[1]
    @classmethod
    def get_paddle_keypoints(cls, env):
        if env.paddle.is_entity:
            pixels = env.parse_object_into_pixels(env.paddle)

            mid_state, eid = pixels[len(pixels) // 2]
            mid_pos = cls.get_entity_pos(mid_state)
            mid = mid_pos[1]
            offset = cls.DEFAULT_PADDLE_SHAPE[0] // 2

            left = mid - offset + 1
            right = mid + offset - 1

            soft_left = left + 1
            soft_right = right - 1


            return left, mid, right, soft_left, soft_right, right, left, left, right

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as dist

def out_of_bounds(r, c, shape):
    return (r < 0 or c < 0 or r >= shape[0] or c >= shape[1])

def shifted(direction, local_program, cell, obs):
    if cell is None:
        new_cell = None
    else:
        new_cell = (cell[0] + direction[0], cell[1] + direction[1])
    return local_program(new_cell, obs)

def cell_is_value(value, cell, obs):
    if cell is None or out_of_bounds(cell[0], cell[1], obs.shape):
        focus = None
    else:
        focus = obs[cell[0], cell[1]]

    return (focus == value)

def at_cell_with_value(value, local_program, obs):
    matches = np.argwhere(obs == value)
    if len(matches) == 0:
        cell = None
    else:
        cell = matches[0]
    return local_program(cell, obs)

def scanning(direction, true_condition, false_condition, cell, obs, max_timeout=50):
    if cell is None:
        return False

    for _ in range(max_timeout):
        cell = (cell[0] + direction[0], cell[1] + direction[1])

        if true_condition(cell, obs):
            return True

        if false_condition(cell, obs):
            return False

        # prevent infinite loops
        if out_of_bounds(cell[0], cell[1], obs.shape):
            return False

    return False

# My classes
class Model(nn.Module):
    # Container for program synthesis model
    def __init__(self, object_types, action_types, num_programs):
        super(Model, self).__init__()

        self.feature_extractor = EntityExtractor()
        self.action_types = action_types
        self.programs = nn.ModuleList()
        for i in range(num_programs):
            self.programs.append(AtActionCell(object_types, action_types))

    def forward(self, env):
        obs = self.feature_extractor.extract(env)
        action_probs = torch.zeros(self.action_types)
        for r in range(obs.shape[0]):
            for c in range(obs.shape[1]):
                for program in self.programs:
                    condition, action = program((r, c), obs)
                    if condition:
                        action_probs[action] += 1

        return dist.Categorical(F.log_softmax(action_probs, dim=0))


class AtActionCell(nn.Module):
    def __init__(self, object_types, action_types):
        super(AtActionCell, self).__init__()
        self.object_types = nn.Parameter(torch.ones(object_types))
        self.positive_object_types = nn.Parameter(torch.ones(object_types))
        self.negative_object_types = nn.Parameter(torch.ones(object_types))

        self.action_types = nn.Parameter(torch.ones(action_types))
        self.direction_types = nn.Parameter(torch.ones(8))
        self.directions = [
            (1, 0), (0, 1),
            (-1, 0), (0, -1),
            (1, 1), (-1, 1),
            (1, -1), (-1, -1)
        ]

    def forward(self, cell, obs):
        # Sample function parameters
        object_probs = F.log_softmax(self.object_types, dim=0)
        positive_object_probs = F.log_softmax(self.positive_object_types, dim=0)
        negative_object_probs = F.log_softmax(self.negative_object_types, dim=0)

        action_probs = F.log_softmax(self.action_types, dim=0)
        direction_probs = F.log_softmax(self.direction_types, dim=0)

        sample_object = dist.Categorical(object_probs).sample()
        sample_positive_object = dist.Categorical(positive_object_probs).sample()
        sample_negative_object = dist.Categorical(negative_object_probs).sample()

        sample_action = dist.Categorical(action_probs).sample()
        sample_direction = dist.Categorical(direction_probs).sample()

        direction = self.directions[sample_direction]

        # Main program
        condition = at_cell_with_value(
            sample_object, 
            lambda cell, obs : scanning(
                direction,
                lambda cell, obs : cell_is_value(sample_positive_object, cell, obs),
                lambda cell, obs : cell_is_value(sample_negative_object, cell, obs),
                cell,
                obs
            ),
            obs
        )

        return condition, sample_action

In [ ]:
import torch.optim as optim

from gym.wrappers import Monitor
from schema_games.breakout.games import StandardBreakout

model = Model(object_types=5, action_types=3, num_programs=10).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

env = Monitor(StandardBreakout(), 'video', force=True)

obs = env.reset()

for epoch in range(10):
    steps = 0
    loss = 0
    while True:
        steps += 1
        env.render()
        action_probs = model(env)
        action = action_probs.sample()
        #action = env.action_space.sample()
        obs, reward, done, _ = env.step(action.item())
        total_reward += reward
        loss += -action_probs.log_prob(action) * reward
        if done:
            optimizer.zero_grad()
            loss = loss / steps
            loss.backward()
            optimizer.step()
            print(loss)
            break

env.close()

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


[---] Lives remaining: 2
[---] Lives remaining: 1
[---] Game over! You lost.
********************************************************************************


RuntimeError: ignored

In [ ]:
env.close()

import io
import base64
from IPython.display import HTML

video = io.open('video/openaigym.video.%s.video000000.mp4' % env.file_infix, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''
    <video width="54" height="auto" alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''
.format(encoded.decode('ascii')))

In [ ]:
torch.save(model.state_dict(), 'rips_sidereal.pt')